In [47]:
import pandas as pd
import requests
from io import StringIO

In [48]:
def get_withdrawn_ipos() -> pd.DataFrame:
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)
        if not tables:
            raise ValueError(f"No tables found.")
        return tables[0]
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [49]:
ipos_withdrawn = get_withdrawn_ipos()
ipos_withdrawn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [50]:
# For class allocation
import numpy as np
conditions = [
    # ipos_withdrawn['Company Name'].str.contains(r'(Corp|Corporation)\b', case=False, regex=True), # if corp we skip it
    ipos_withdrawn['Company Name'].str.contains(r'\b(Inc|Incorporated)\b', case=False, regex=True),
    ipos_withdrawn['Company Name'].str.contains(r'\bGroup\b', case=False, regex=True),
    ipos_withdrawn['Company Name'].str.contains(r'\b(Ltd|Limited)\b', case=False, regex=True),
    ipos_withdrawn['Company Name'].str.contains(r'\bHoldings\b', case=False, regex=True),
]
choices = ['Inc', 'Group', 'Limited', 'Holdings']
# Apply the logic
ipos_withdrawn['Company Class'] = np.select(conditions, choices, default='Other')
# Special case (custom logic): Acquisition Corp/Corporation -> Acq.Corp
mask_acq = ipos_withdrawn['Company Name'].str.contains(r'\b(Corp|Corporation)\b', case=False, regex=True)
ipos_withdrawn.loc[mask_acq, 'Company Class'] = ipos_withdrawn.loc[mask_acq, 'Company Name'].str[:3] + '.Corp'

<ipython-input-50-3043089604>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ipos_withdrawn['Company Name'].str.contains(r'\b(Inc|Incorporated)\b', case=False, regex=True),
<ipython-input-50-3043089604>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ipos_withdrawn['Company Name'].str.contains(r'\b(Ltd|Limited)\b', case=False, regex=True),
<ipython-input-50-3043089604>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_acq = ipos_withdrawn['Company Name'].str.contains(r'\b(Corp|Corporation)\b', case=False, regex=True)


In [52]:
ipos_withdrawn[['Company Name','Company Class']][:5]

,Company Name,Company Class
0,"Odyssey Therapeutics, Inc.",Inc
1,"Unifoil Holdings, Inc.",Inc
2,"Aurion Biotech, Inc.",Inc
3,"PHI Group, Inc.",Inc
4,One Power Company,Other


In [53]:
ipos_withdrawn['Price Range'][:5]

,Price Range
0,-
1,$3.00 - $4.00
2,-
3,-
4,-


In [54]:
# Clean price
def avg_price(price_str):
    if not isinstance(price_str, str) or price_str.strip() == '-':
        return None
    # Remove $ and whitespace
    clean = price_str.replace('$', '').strip()
    # Handle ranges
    if '-' in clean:
        try:
            low, high = map(float, clean.split('-'))
            return (low + high) / 2
        except ValueError:
            return None
    else:
        try:
            return float(clean)
        except ValueError:
            return None

In [55]:
ipos_withdrawn['Avg. Price'] = ipos_withdrawn['Price Range'].apply(avg_price)
ipos_withdrawn.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN


In [56]:
# Step-by-step
ipos_withdrawn['Shares Offered'] = (
    ipos_withdrawn['Shares Offered']
    .replace('-', np.nan)                     # Replace '-' with NaN
    .astype(str)                              # Convert to string to ensure strip works
    .str.replace(',', '', regex=False)        # Remove commas if present (e.g., '1,000')
    .str.strip()                              # Remove leading/trailing whitespace
)

# Convert to numeric safely (invalid parsing becomes NaN)
ipos_withdrawn['Shares Offered'] = pd.to_numeric(ipos_withdrawn['Shares Offered'], errors='coerce')

In [57]:
ipos_withdrawn.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN
4,ONE,One Power Company,-,NaN,Other,NaN


In [58]:
ipos_withdrawn['Withdraw Value'] = ipos_withdrawn['Avg. Price'] * ipos_withdrawn['Shares Offered']
ipos_withdrawn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          100 non-null    object 
 1   Company Name    100 non-null    object 
 2   Price Range     100 non-null    object 
 3   Shares Offered  72 non-null     float64
 4   Company Class   100 non-null    object 
 5   Avg. Price      73 non-null     float64
 6   Withdraw Value  71 non-null     float64
dtypes: float64(3), object(4)
memory usage: 5.6+ KB


In [59]:
print(ipos_withdrawn.groupby('Company Class')['Withdraw Value'].sum().sort_values(ascending=False).idxmax())

Inc


In [60]:
print(ipos_withdrawn.groupby('Company Class')['Withdraw Value'].sum().sort_values(ascending=False)[0])

2257164204.85


<ipython-input-60-857333082>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(ipos_withdrawn.groupby('Company Class')['Withdraw Value'].sum().sort_values(ascending=False)[0])


In [61]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [63]:
ipos_2024 = get_ipos_by_year(2024)
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], errors='coerce')
ipos_2024 = ipos_2024[ipos_2024['IPO Date'] < pd.Timestamp('2024-06-01')]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, 148 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      77 non-null     datetime64[ns]
 1   Symbol        77 non-null     object        
 2   Company Name  77 non-null     object        
 3   IPO Price     77 non-null     object        
 4   Current       77 non-null     object        
 5   Return        77 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.2+ KB


In [68]:
ipos_2024 = ipos_2024[ipos_2024['IPO Price']!='-']
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 149 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      75 non-null     datetime64[ns]
 1   Symbol        75 non-null     object        
 2   Company Name  75 non-null     object        
 3   IPO Price     75 non-null     object        
 4   Current       75 non-null     object        
 5   Return        75 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.1+ KB


In [69]:
import time
import yfinance as yf

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ipos_2024['Symbol']):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)

  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [70]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [71]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23229 entries, 0 to 23228
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Open                           23229 non-null  float64
 1   High                           23229 non-null  float64
 2   Low                            23229 non-null  float64
 3   Close                          23229 non-null  float64
 4   Volume                         23229 non-null  int64  
 5   Dividends                      23229 non-null  float64
 6   Stock Splits                   23229 non-null  float64
 7   Ticker                         23229 non-null  object 
 8   Year                           23229 non-null  int32  
 9   Month                          23229 non-null  int32  
 10  Weekday                        23229 non-null  int32  
 11  Date                           23229 non-null  object 
 12  growth_1d                      23154 non-null 

In [72]:
# Ensure 'Date' is in datetime format (if not already)
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], errors='coerce')

# Filter for the specific trading day
stocks_df_20250606 = stocks_df[stocks_df['Date'] == pd.Timestamp('2025-06-06')]

# Check the result
print(stocks_df_20250606.head())

           Open       High        Low      Close   Volume  Dividends  \
259   36.439999  36.540001  35.930000  36.389999   137900        0.0   
531   20.430000  20.440001  19.969999  20.410000     1300        0.0   
770   10.510000  10.510000  10.510000  10.510000        0        0.0   
1043   0.380000   0.386000   0.201000   0.295000  3428900        0.0   
1317   1.110000   1.270000   1.100000   1.255000   501000        0.0   

      Stock Splits Ticker  Year  Month  ...  growth_252d growth_365d  \
259            0.0    BOW  2025      6  ...     1.442331         NaN   
531            0.0    HDL  2025      6  ...     1.007155         NaN   
770            0.0   RFAI  2025      6  ...          NaN         NaN   
1043           0.0   JDZG  2025      6  ...     0.168571         NaN   
1317           0.0    RAY  2025      6  ...     0.343459         NaN   

      growth_future_30d    SMA10     SMA20  growing_moving_average  \
259                 NaN  36.9540  37.70400                      

In [73]:
stocks_df_20250606[['growth_252d', 'Sharpe']].describe()

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152898,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


In [83]:
for m in range(1, 13):
    days = m * 21
    stocks_df[f'future_growth_{m}m'] = stocks_df.groupby('Ticker')['Close'].transform(
        lambda x: x.shift(-days) / x
    )

In [84]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23229 entries, 0 to 23228
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Open                           23229 non-null  float64       
 1   High                           23229 non-null  float64       
 2   Low                            23229 non-null  float64       
 3   Close                          23229 non-null  float64       
 4   Volume                         23229 non-null  int64         
 5   Dividends                      23229 non-null  float64       
 6   Stock Splits                   23229 non-null  float64       
 7   Ticker                         23229 non-null  object        
 8   Year                           23229 non-null  int32         
 9   Month                          23229 non-null  int32         
 10  Weekday                        23229 non-null  int32         
 11  Date           

In [85]:
min_dates = stocks_df.groupby('Ticker')['Date'].min().reset_index()
min_dates.columns = ['Ticker', 'min_date']

In [86]:
min_dates.head()

,Ticker,min_date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22


In [87]:
min_dates_renamed = min_dates.rename(columns={'min_date': 'Date'})

In [88]:
ipo_growth_df = pd.merge(
    min_dates_renamed,
    stocks_df,
    on=['Ticker', 'Date'],
    how='inner'
)

In [89]:
ipo_growth_df.head()

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,AHR,2024-02-07,12.085784,12.471401,11.878868,12.433780,12732800,0.0,0.0,2024,...,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257204,2.259752,2.273312
1,ALAB,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083


In [90]:
ipo_growth_df.describe()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,75,75.000000,75.000000,75.000000,75.000000,7.500000e+01,75.0,75.0,75.0,75.0000,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,2024-03-24 07:40:48,15.618140,17.843530,14.309872,15.856623,5.341061e+06,0.0,0.0,2024.0,3.2800,...,0.833825,0.825087,0.803769,0.864186,0.847150,0.832984,0.881780,0.917946,0.882535,0.900862
min,2024-01-09 00:00:00,0.010700,0.015700,0.006200,0.011600,6.000000e+02,0.0,0.0,2024.0,1.0000,...,0.060947,0.045368,0.054109,0.061432,0.044086,0.043103,0.033144,0.037769,0.023674,0.038947
25%,2024-02-08 12:00:00,4.770000,5.700000,3.602500,4.140000,7.332000e+05,0.0,0.0,2024.0,2.0000,...,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,2024-03-28 00:00:00,10.000000,10.320000,10.000000,10.040000,1.992600e+06,0.0,0.0,2024.0,3.0000,...,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.634667
75%,2024-04-25 00:00:00,21.875000,23.430000,19.125000,19.900000,5.451900e+06,0.0,0.0,2024.0,4.0000,...,1.069085,1.134300,1.016381,1.093948,1.114468,1.082457,1.049719,1.200830,1.106780,1.136392
max,2024-09-11 00:00:00,103.250000,103.500000,95.029999,98.000000,4.870550e+07,0.0,0.0,2024.0,9.0000,...,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711
std,NaN,17.546428,19.276158,16.692335,18.235333,8.882742e+06,0.0,0.0,0.0,1.6649,...,0.409763,0.401969,0.488349,0.653180,0.712944,0.762423,0.936957,0.911431,0.863183,0.892906


In [91]:
cols = [f'future_growth_{m}m' for m in range(1, 13)]

# Step 2: Compute mean returns for each holding period
mean_returns = ipo_growth_df[cols].mean()

# Step 3: Find the best one
best_month = mean_returns.idxmax()
best_value = mean_returns.max()

# Step 4: Check the uplift and <2 condition
uplift_check = all((best_value - mean_returns.drop(best_month)) > 0.01)
below_double = best_value < 2

# Step 5: Report
if uplift_check and below_double:
    print(f"✅ Best holding period: {best_month} with mean return = {best_value:.4f}")
else:
    print("❌ No holding period satisfies both conditions.")
    print(mean_returns.sort_values(ascending=False))  # Still useful to see

✅ Best holding period: future_growth_2m with mean return = 0.9405


In [30]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")


Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=f1ff75fb-6f29-49a0-a2de-bf05f4fadc69
To: /content/data.parquet
100%|██████████| 130M/130M [00:02<00:00, 65.0MB/s]


In [31]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [32]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [33]:
net_income

np.float64(24295.523125248386)